In [1]:
import sys
import sqlite3
import numpy as np
import os
import argparse
import itertools
import collections

IS_PYTHON3 = sys.version_info[0] >= 3

MAX_IMAGE_ID = 2**31 - 1

CREATE_CAMERAS_TABLE = """CREATE TABLE IF NOT EXISTS cameras (
    camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    model INTEGER NOT NULL,
    width INTEGER NOT NULL,
    height INTEGER NOT NULL,
    params BLOB,
    prior_focal_length INTEGER NOT NULL)"""

CREATE_DESCRIPTORS_TABLE = """CREATE TABLE IF NOT EXISTS descriptors (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)"""

CREATE_IMAGES_TABLE = """CREATE TABLE IF NOT EXISTS images (
    image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name TEXT NOT NULL UNIQUE,
    camera_id INTEGER NOT NULL,
    prior_qw REAL,
    prior_qx REAL,
    prior_qy REAL,
    prior_qz REAL,
    prior_tx REAL,
    prior_ty REAL,
    prior_tz REAL,
    CONSTRAINT image_id_check CHECK(image_id >= 0 and image_id < {}),
    FOREIGN KEY(camera_id) REFERENCES cameras(camera_id))
""".format(MAX_IMAGE_ID)

CREATE_TWO_VIEW_GEOMETRIES_TABLE = """
CREATE TABLE IF NOT EXISTS two_view_geometries (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    config INTEGER NOT NULL,
    F BLOB,
    E BLOB,
    H BLOB,
    qvec BLOB,
    tvec BLOB)
"""

CREATE_KEYPOINTS_TABLE = """CREATE TABLE IF NOT EXISTS keypoints (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)
"""

CREATE_MATCHES_TABLE = """CREATE TABLE IF NOT EXISTS matches (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB)"""

CREATE_NAME_INDEX = \
    "CREATE UNIQUE INDEX IF NOT EXISTS index_name ON images(name)"

CREATE_ALL = "; ".join([
    CREATE_CAMERAS_TABLE,
    CREATE_IMAGES_TABLE,
    CREATE_KEYPOINTS_TABLE,
    CREATE_DESCRIPTORS_TABLE,
    CREATE_MATCHES_TABLE,
    CREATE_TWO_VIEW_GEOMETRIES_TABLE,
    CREATE_NAME_INDEX
])

from database import image_ids_to_pair_id, pair_id_to_image_ids, array_to_blob, blob_to_array, COLMAPDatabase

from read_write_model import read_model, CAMERA_MODEL_NAMES

In [2]:
input_model = "../../../colmap_dataset/gerrard-hall-small-backup/sparse/"
input_format = ".txt" 
read_database = "../../../colmap_dataset/gerrard-hall-small-backup/database.db"
database_path = "../../../colmap_dataset/gerrard-hall-small/database.db"
delete = True

In [3]:
if delete:
    if os.path.exists(database_path):
        print("Deleting")
        os.remove(database_path)

Deleting


In [4]:
cameras_out, images_out, points3D_out = read_model(path=input_model, ext=input_format)

if os.path.exists(database_path):
    print("ERROR: database path already exists -- will not modify it.")
    assert(False)

# Open the database.

db_to_read = COLMAPDatabase.connect(read_database, read_only=True)
db = COLMAPDatabase.connect(database_path)

# For convenience, try creating all the tables upfront.
db.create_tables()

In [5]:
import new_generate as ng

In [6]:
# Cameras = ng.CAMERAS(db_to_read, cameras_out)
# Descriptors = ng.DESCRIPTORS(db_to_read)
# Images = ng.IMAGES(db_to_read, images_out)
# Two_view_geometrys = ng.TWO_VIEW_GEOMETRYS(db_to_read, images_out, True)
# Keypoints = ng.KEYPOINTS(db_to_read)
# Matches = ng.MATCHES(db_to_read)

In [7]:
# Keypoints.generate_GT_inliers(Two_view_geometrys, Images, Cameras, points3D_out, 1)

In [8]:
# increase_ratio = 1.1
# outlier_ratio = 0.5
# max_iter_outlier = 100
# max_inlier_error = 2

In [9]:
# all_points3D = np.zeros((len(points3D_out), 3))
# for num_point3D, point3D in enumerate(points3D_out.values()):
#     all_points3D[num_point3D, :] = point3D.xyz
# mini3D = np.min(all_points3D, axis=0) * increase_ratio
# maxi3D = np.max(all_points3D, axis=0) * increase_ratio

# GT_outliers = []
# pt3D_outliers = []

# for image_id, keypoint in Keypoints._keypoints.items():
#     image = Images._images[image_id]
#     camera = Cameras._cameras[image._camera_id]
#     width = camera._width
#     height = camera._height
#     num_inlier = len(keypoint._corrected)
#     if outlier_ratio < 1:
#         num_outlier_req = np.floor(outlier_ratio * num_inlier / (1 - outlier_ratio))
#     else:
#         num_outlier_req = np.floor(outlier_ratio)
    
#     num_outlier_created = 0
#     iteration = 0
#     while num_outlier_created < num_outlier_req and iteration < num_outlier_req + max_iter_outlier:
#         xyz_val = np.random.uniform(mini3D, maxi3D)
#         uv_val = Keypoints._align(xyz_val, camera, image, False)
#         u, v = uv_val
#         if u > 0 and v > 0 and u < width and v < height:
#             min_offset = np.sqrt(max_inlier_error)
            
#             theta1 = np.arctan((height - v) / (width - u))
#             theta2 = np.arctan(u / (height - v)) + np.pi / 2
#             theta3 = np.arctan(v / u) + np.pi
#             theta4 = np.arctan((width - u) / v) + np.pi * 3 / 2
            
#             assert(theta1 > 0 and theta1 < np.pi / 2)
#             assert(theta2 > np.pi / 2 and theta2 < np.pi)
#             assert(theta3 > np.pi and theta3 < np.pi * 3 / 2)
#             assert(theta4 > np.pi * 3 / 2 and theta4 < 2 * np.pi)
            
#             theta = np.random.uniform(0, 2 * np.pi)
            
#             if theta >= theta4 or theta < theta1:
#                 max_offset = (width - u) / np.cos(theta)
#             elif theta >= theta1 and theta < theta2:
#                 max_offset = (height - v) / np.cos(theta - np.pi / 2)
#             elif theta >= theta2 and theta < theta3:
#                 max_offset = u / np.cos(theta - np.pi)
#             elif theta >= theta3 and theta < theta4:
#                 max_offset = v / np.cos(theta - np.pi * 3 / 2)
                
#             if max_offset >= min_offset:
#                 offset = np.random.uniform(min_offset, max_offset)
                
#                 u += np.cos(theta) * offset
#                 v += np.sin(theta) * offset
                
#                 assert(u >= 0 and u <= width)
#                 assert(v >= 0 and v <= height)
                
#                 GT_outliers.append(np.array([u, v]))
#                 pt3D_outliers.append(xyz_val)
                
#                 num_outlier_created += 1
#         iteration += 1

## Load data

In [10]:
Cameras = ng.CAMERAS(db_to_read, cameras_out)
Images = ng.IMAGES(db_to_read, images_out)

In [11]:
Two_view_geometrys = ng.TWO_VIEW_GEOMETRYS(db_to_read, images_out, True)

In [12]:
Matches = ng.MATCHES(db_to_read)

In [13]:
from collections import defaultdict

In [14]:
image_right = defaultdict(lambda: defaultdict(list))
image_left = defaultdict(lambda: defaultdict(list))
for id_pair, two_view in Two_view_geometrys._two_view_geometries.items():
    image_id1, image_id2 = two_view._id_pair
    for id_feature1, id_feature2 in two_view._data:
        image_right[image_id1][id_feature1].append([image_id2, id_feature2])
        image_left[image_id2][id_feature2].append([image_id1, id_feature1])

In [15]:
Keypoints = ng.KEYPOINTS(db_to_read)

In [16]:
Descriptors = ng.DESCRIPTORS(db_to_read)

In [17]:
class FEATURE:
    def __init__(self,
                 feature_id_, image_id_, point3D_id_, index_,
                 position_, descriptor_,
                 match_right_, match_left_,
                 ncols_keypoints_, ncols_descriptors_):
        self._feature_id = feature_id_
        self._image_id = image_id_
        self._point3D_id = point3D_id_
        self._index = index_
        
        self._position = position_
        self._descriptor = descriptor_
        self._match_right = match_right_
        self._match_left = match_left_
        
        self._ncols_keypoints = ncols_keypoints_
        self._ncols_descriptors = ncols_descriptors_
        
    def is_valid(self):
        return len(self._match_right) + len(self._match_left) > 0

In [18]:
features_by_image = defaultdict(dict)
all_features = {}

In [19]:
id_feature = 0
for image_id, keypoints in Keypoints._keypoints.items():
    for index in range(keypoints._rows):
        point3D_id = Images._images[image_id]._image_out.point3D_ids[index]
        descriptor = Descriptors._descriptors[image_id]
        match_right = {}
        if image_id in image_right:
            if index in image_right[image_id]:
                match_right = image_right[image_id][index]
        match_left = {}
        if image_id in image_left:
            if index in image_left[image_id]:
                match_left = image_left[image_id][index]
        feature = FEATURE(id_feature, image_id, point3D_id, index, 
                          keypoints._data[index, :], descriptor._data[index, :],
                          match_right, match_left,
                          keypoints._cols, descriptor._cols
                         )
        features_by_image[image_id][index] = feature
        all_features[id_feature] = feature
        id_feature += 1

In [20]:
keep = {}
for image_id, features in features_by_image.items():
    keep[image_id] = set()
    for feature in features.values():
        if feature.is_valid():
            keep[image_id].add(feature._feature_id)

In [21]:
match_to_right = defaultdict(list)
match_to_left = defaultdict(list)
for id_pair, two_view in Two_view_geometrys._two_view_geometries.items():
    image_id1, image_id2 = two_view._id_pair
    for id_feature1, id_feature2 in two_view._data:
        feature1 = features_by_image[image_id1][id_feature1]
        feature2 = features_by_image[image_id2][id_feature2]
        match_to_right[feature1._feature_id].append(feature2._feature_id)
        match_to_left[feature2._feature_id].append(feature1._feature_id)

In [22]:
for feature_id, feature in all_features.items():
    feature._match_right = match_to_right[feature_id]
    feature._match_left = match_to_left[feature_id]

## Process data

In [23]:
def _align(xyz_val_, camera_, image_, safe_ = True):
    uv_val = camera_._calib.dot(image_._rotation.dot(xyz_val_) + image_._translation)
    if uv_val[2] > 0 or (uv_val[2] < 0 and not safe_):
        uv_val = uv_val / uv_val[2]
    else:
        print("ERROR: point not in front of camera")

    return uv_val[:2]

In [24]:
def _add_noise(uv_val_, noise_std_, camera_):
    noisy_uv_val = uv_val_ + np.random.uniform(-noise_std_, noise_std_, 2)
    truncated = np.clip(noisy_uv_val, a_min = [0, 0], a_max = [camera_._width, camera_._height])
    return truncated

In [25]:
noise_std = 2

In [26]:
for image_id, features_to_keep in keep.items():
    image = Images._images[image_id]
    camera = Cameras._cameras[image._camera_id]
    for feature_id in features_to_keep:
        feature = all_features[feature_id]
        xyz_val = points3D_out[feature._point3D_id].xyz
        uv_val = _align(xyz_val, camera, image)
        if noise_std > 0:
            uv_val = _add_noise(uv_val, noise_std, camera)
        feature._position[:2] = uv_val

In [27]:
increase_ratio = 1.1
outlier_ratio = 0.35
max_try_outlier = 100
max_inlier_error = 2

In [28]:
def generate_outlier(xyz_val_, camera_, image_, max_inlier_error_, iteration_, max_iter_outlier_):
    width = camera._width
    height = camera._height
    
    uv_val = _align(xyz_val_, camera, image, False)
    u, v = uv_val
    
    if u > 0 and v > 0 and u < width and v < height:
        min_offset = np.sqrt(max_inlier_error_)

        theta1 = np.arctan((height - v) / (width - u))
        theta2 = np.arctan(u / (height - v)) + np.pi / 2
        theta3 = np.arctan(v / u) + np.pi
        theta4 = np.arctan((width - u) / v) + np.pi * 3 / 2

        assert(theta1 > 0 and theta1 < np.pi / 2)
        assert(theta2 > np.pi / 2 and theta2 < np.pi)
        assert(theta3 > np.pi and theta3 < np.pi * 3 / 2)
        assert(theta4 > np.pi * 3 / 2 and theta4 < 2 * np.pi)
        while iteration_ < max_iter_outlier_: 

            theta = np.random.uniform(0, 2 * np.pi)

            if theta >= theta4 or theta < theta1:
                max_offset = (width - u) / np.cos(theta)
            elif theta >= theta1 and theta < theta2:
                max_offset = (height - v) / np.cos(theta - np.pi / 2)
            elif theta >= theta2 and theta < theta3:
                max_offset = u / np.cos(theta - np.pi)
            elif theta >= theta3 and theta < theta4:
                max_offset = v / np.cos(theta - np.pi * 3 / 2)

            if max_offset >= min_offset:
                offset = np.random.uniform(min_offset, max_offset)

                u += np.cos(theta) * offset
                v += np.sin(theta) * offset

                assert(u >= 0 and u <= width)
                assert(v >= 0 and v <= height)
                
                iteration_ += 1
                return np.array([u, v])
            iteration_ += 1
    iteration_ += 1
    return None

In [29]:
two_views_withfeatures = {}
for id_pair, two_view in Two_view_geometrys._two_view_geometries.items():
    image_id1, image_id2 = two_view._id_pair
    two_views_matchs = []
    for id_feature1, id_feature2 in two_view._data:
        feature1 = features_by_image[image_id1][id_feature1]
        feature2 = features_by_image[image_id2][id_feature2]
        two_views_matchs.append([feature1._feature_id, feature2._feature_id])
    two_views_withfeatures[id_pair] = two_views_matchs

In [30]:
for i, e in keep.items():
    print(i, len(e))

1 2768
2 2612
3 2949
4 2345
5 1914
6 2497
7 1946
8 2640
9 2620


In [31]:
id_feature_new = len(all_features)
for id_pair, two_view in two_views_withfeatures.items():
    image_ids = pair_id_to_image_ids(id_pair)
    if 1 in image_ids and 9 in image_ids:
        continue
    elif 1 in image_ids:
        side = 1
    elif 9 in image_ids:
        side = 0
    else:
        side = np.random.randint(0, 2)
        
    num_matches = len(two_view)
    if outlier_ratio < 1:
        num_outlier_req = np.floor(outlier_ratio * num_matches).astype(int)
    else:
        num_outlier_req = np.floor(outlier_ratio).astype(int)
    
    outliers_idx = np.random.choice(num_matches, num_outlier_req, replace=False)
    
    other_side = 1 - side
    image_id = image_ids[side]
    image = Images._images[image_id]
    camera = Cameras._cameras[image._camera_id]

    iteration = 0
    for outlier_id in outliers_idx:
        match_to_modify = two_view[outlier_id]
        feature_to_modify = all_features[match_to_modify[side]]
        feature_unchanged = all_features[match_to_modify[other_side]]
        
        xyz_val = points3D_out[feature_to_modify._point3D_id].xyz
        outlier_pos = generate_outlier(xyz_val, camera, image, 
                                       max_inlier_error, iteration, max_try_outlier + num_outlier_req)
        
        if outlier_pos is not None:
            position = feature_to_modify._position
            position[:2] = outlier_pos
            
            if side:
                idx_to_change_side = \
                    np.where(np.array(feature_to_modify._match_left) == feature_unchanged._feature_id)[0][0]
                feature_to_modify._match_left.pop(idx_to_change_side)
                
                idx_to_change_otherside =  \
                    np.where(np.array(feature_unchanged._match_right) == feature_to_modify._feature_id)[0][0]
                feature_unchanged._match_right[idx_to_change_otherside] = id_feature_new
                
                match_right = []
                match_left = [feature_unchanged._feature_id]                
            else:
                idx_to_change_side =  \
                    np.where(np.array(feature_to_modify._match_right) == feature_unchanged._feature_id)[0][0]

                feature_to_modify._match_right.pop(idx_to_change_side)
                
                idx_to_change_otherside =  \
                    np.where(np.array(feature_unchanged._match_left) == feature_to_modify._feature_id)[0][0]
                feature_unchanged._match_left[idx_to_change_otherside] = id_feature_new
                
                match_right = [feature_unchanged._feature_id]
                match_left = []
            
            new_feature = FEATURE(id_feature_new, image_id, feature_to_modify._point3D_id, -1, 
                          position, feature_to_modify._descriptor,
                          match_right, match_left,
                          feature_to_modify._ncols_keypoints, feature_to_modify._ncols_descriptors
                         )
            
            all_features[id_feature_new] = new_feature
            keep[image_id].add(id_feature_new)
            id_feature_new += 1

## Write data

In [32]:
descriptors_feated = defaultdict(list)
keypoints_feated = defaultdict(list)
matches_feated = defaultdict(list)
two_view_geometry_feated = defaultdict(list)
new_feature_by_image = defaultdict(dict)
for image_id, features_to_keep in keep.items():
    for index_feat, feature_id in enumerate(features_to_keep):
        feature = all_features[feature_id]
        feature._index = index_feat
        new_feature_by_image[image_id][index_feat] = feature
        descriptors_feated[image_id].append([feature._descriptor, feature._ncols_descriptors])
        keypoints_feated[image_id].append([feature._position, feature._ncols_keypoints])

In [33]:
for image_id, features_to_keep in keep.items():
    for index_feat, feature_id in enumerate(features_to_keep):
        feature = all_features[feature_id]
        for feature_id2 in feature._match_right:
            feature2 = all_features[feature_id2]
            image_id1 = feature._image_id
            image_id2 = feature2._image_id
            index1 = feature._index
            index2 = feature2._index
            if image_id1 > image_id2:
                value = [index2, index1]
            else:
                value = [index1, index2]
            pair_id = image_ids_to_pair_id(image_id1, image_id2)
        if value not in two_view_geometry_feated[pair_id]:
            two_view_geometry_feated[pair_id].append(value)
        if value not in matches_feated[pair_id]:
            matches_feated[pair_id].append(value)
        for feature_id0 in feature._match_left:
            feature0 = all_features[feature_id0]
            image_id1 = feature._image_id
            image_id0 = feature0._image_id
            index1 = feature._index
            index0 = feature0._index
            value = [index0, index1]
            if image_id0 > image_id1:
                value = [index1, index0]
            else:
                value = [index0, index1]
            pair_id = image_ids_to_pair_id(image_id0, image_id1)
            if value not in two_view_geometry_feated[pair_id]:
                two_view_geometry_feated[pair_id].append(value)
            if value not in matches_feated[pair_id]:
                matches_feated[pair_id].append(value)

In [34]:
descriptors_feated_curated = {}
for image_id, descriptors in descriptors_feated.items():
    num_row = len(descriptors)
    num_col = descriptors[0][1]
    descript_as_array = np.zeros((num_row, num_col))
    for index, descriptor in enumerate(descriptors):
        if descriptor[1] != num_col:
            print("ERROR")
        else:
            descript_as_array[index, :] = descriptor[0]
    descriptors_feated_curated[image_id] = ng.DESCRIPTOR(image_id, 
                                                         num_row, num_col, 
                                                         array_to_blob(descript_as_array.astype(np.int8)))

In [35]:
keypoints_feated_curated = {}
for image_id, keypoints in keypoints_feated.items():
    num_row = len(keypoints)
    num_col = keypoints[0][1]
    keypoint_as_array = np.zeros((num_row, num_col))
    for index, keypoint in enumerate(keypoints):
        if keypoint[1] != num_col:
            print("ERROR")
        else:
            keypoint_as_array[index, :] = keypoint[0]
    keypoints_feated_curated[image_id] = ng.KEYPOINT(image_id,
                                                     num_row, num_col,
                                                     array_to_blob(keypoint_as_array.astype(np.float32)))

In [36]:
matches_feated_curated = {}
for pair_id, matches in matches_feated.items():
    num_row = len(matches)
    num_col = 2
    match_as_array = np.array(matches)
    matches_feated_curated[pair_id] = ng.MATCH(pair_id,
                                               num_row, num_col,
                                               array_to_blob(match_as_array.astype(np.int32)))

In [37]:
twoview_feated_curated = {}
for pair_id, two_view in two_view_geometry_feated.items():
    num_row = len(two_view)
    num_col = 2
    match_as_array = np.array(two_view, dtype=np.int32)
    two_view_init = Two_view_geometrys._two_view_geometries[pair_id]
    twoview_feated_curated[pair_id] = ng.TWO_VIEW_GEOMETRY(pair_id,
                                               num_row, num_col,
                                               match_as_array,
                                               two_view_init._config,
                                               two_view_init._F, two_view_init._E, two_view_init._H,
                                               two_view_init._qvec, two_view_init._tvec)

In [38]:
len(keypoints_feated_curated[1]._data)

2768

In [39]:
for i, e in keep.items():
    print(i, len(e))

1 2768
2 5255
3 4251
4 3950
5 3465
6 4050
7 2678
8 4959
9 2620


In [40]:
len(Keypoints._keypoints[1]._data)

10872

In [41]:
for camera_id in Cameras._cameras:
    Cameras._cameras[camera_id].write(db)

In [42]:
for image_id in Images._images:
    Images._images[image_id].write(db)

In [43]:
for image_id in descriptors_feated_curated:
    descriptors_feated_curated[image_id].write(db)

In [44]:
for image_id in keypoints_feated_curated:
    keypoints_feated_curated[image_id].write(db)

In [45]:
for pair_id in twoview_feated_curated:
    twoview_feated_curated[pair_id].write(db)

In [46]:
for pair_id in matches_feated_curated:
    matches_feated_curated[pair_id].write(db)

In [47]:
# Commit the data to the file.
db.commit()

# Clean up.
db.close()